Version: 02.14.2023

# ラボ 2.1: 機械学習の NLP 問題への適用

このラボでは、Amazon SageMaker の組み込み機械学習 (ML) モデル (__LinearLearner__) を使って、レビューデータセットの __isPositive__ フィールドを予測します。

## ビジネスシナリオの紹介
あなたはオンライン小売店で働いており、ネガティブなレビューを投稿した顧客のカスタマーエンゲージメントを向上させたいと思っています。この会社では、ネガティブなレビューを検出し、検出したレビューをカスタマーサービスエージェントに割り当てて対処したいと考えています。

あなたは機械学習を使ってネガティブなレビューを検出し、この問題の一部を解決するタスクが割り当てられました。ポジティブまたはネガティブに分類されたレビューを含むデータセットへのアクセス権が与えられています。このデータセットを使って機械学習モデルをトレーニングし、新しいレビューの感情を予測します。

## このデータセットについて
[AMAZON-REVIEW-DATA-CLASSIFICATION.csv](https://github.com/aws-samples/aws-machine-learning-university-accelerated-nlp/tree/master/data/examples) ファイルには、商品の実際のレビューが含まれており、レビューにはテキストデータと数値データの両方が含まれています。各レビューには、「_positive (1)_」または「_negative (0)_」というラベルが付けられています。

データセットには次の特徴が含まれています。
* __reviewText:__ レビューのテキスト
* __summary:__ レビューの概要
* __verified:__ 購入が確認されたかどうか (True または False)
* __time:__ レビューの Unix タイムスタンプ
* __log_votes:__ 対数調整された投票ログ (1+ 投票)
* __isPositive:__ レビューがポジティブかネガティブか (1 または 0)

このラボのデータセットは Amazon の許可のもと提供されており、Amazon License and Access (https://www.amazon.com/gp/help/customer/display.html?nodeId=201909000) の規約に準じます。本コースの実施以外の目的でこのデータセットをコピー、変更、販売、エクスポート、使用することは明示的に禁止されています。

## ラボのステップ

このラボを完了するには、次のステップを実行します。

1. [データセットを読み込む](#1.-Reading-the-dataset)
2. [探索的データ分析を行う](#2.-Performing-exploratory-data-analysis)
3. [テキスト処理: ストップワードを削除し語幹解釈を実行する](#3.-Text-processing:-Removing-stopwords-and-stemming)
4. [トレーニング、検証、テストデータを分割する](#4.-Splitting-training,-validation,-and-test-data)
5. [パイプラインと ColumnTransformer でデータを処理する](#5.-Processing-data-with-pipelines-and-a-ColumnTransformer)
6. [組み込み SageMaker アルゴリズムで分類器をトレーニングする](#6.-Training-a-classifier-with-a-built-in-SageMaker-algorithm)
7. [モデルを評価する](#7.-Evaluating-the-model)
8. [モデルをエンドポイントにデプロイする](#8.-Deploying-the-model-to-an-endpoint)
9. [エンドポイントをテストする](#9.-Testing-the-endpoint)
10. [モデルアーティファクトをクリーンアップする](#10.-Cleaning-up-model-artifacts)
    
## 作業内容を送信する

1.ラボコンソールで [**送信**] をクリックして進行状況を記録し、メッセージが表示されたら [**はい**] をクリックします。

1.数分経っても結果が表示されない場合は、この手順の上部に戻り、[**Grades**] をクリックします。

     **ヒント**: 作業内容は複数回送信できます。作業内容を変更したら、もう一度 [**送信**] をクリックします。最後に送信したものがこのラボの記録として残ります。

1.作業内容に関する詳細なフィードバックを参照するには、[**詳細**]、[**View Submission Report**] の順にクリックします。 

まず、pip、sagemaker、scikit-learn をインストールまたはアップグレードします。

[scikit-learn](https://scikit-learn.org/stable/) はオープンソースの機械学習ライブラリです。モデル適合、データの前処理、モデルの選択と評価といった、他種多様なユーティリティツールを提供します。

In [ ]:
#Upgrade dependencies
!pip install --upgrade pip
!pip install --upgrade scikit-learn
!pip install --upgrade sagemaker
!pip install --upgrade botocore
!pip install --upgrade awscli

## 1.データセットを読み込む
([先頭に戻る](#Lab-2.1:-Applying-ML-to-an-NLP-Problem))

__pandas__ ライブラリを使ってデータセットを読み込みます。[Pandas] (https://pandas.pydata.org/pandas-docs/stable/index.html) は、データ分析によく使われる python ライブラリです。データ操作、クリーニング、データラングリング、視覚化の機能を備えています。

In [ ]:
import pandas as pd

df = pd.read_csv('../data/AMAZON-REVIEW-DATA-CLASSIFICATION.csv')

print('The shape of the dataset is:', df.shape)

データセットの最初の 5 行を見てください。

In [ ]:
df.head(5)

ノートブックのオプションを変更して、より多くのテキストデータを表示できます。

In [ ]:
pd.options.display.max_rows
pd.set_option('display.max_colwidth', None)
df.head()

必要に応じて、特定のエントリを確認できます。

In [ ]:
print(df.loc[[580]])

自分が扱っているデータタイプを確認することをお勧めします。DataFrame で  `dtypes` を使ってタイプを表示できます。

In [ ]:
df.dtypes

## 2.探索的データ分析を行う
([先頭に戻る](#Lab-2.1:-Applying-ML-to-an-NLP-Problem))

次に、データセットのターゲット分布を確認します。

In [ ]:
df['isPositive'].value_counts()

ビジネス上の問題は、ネガティブなレビュー (_0_) を見つけることに焦点を当てています。しかし、線形学習者のモデルチューニングでは、デフォルトで正の値 (_1_) が検索されます。負の値 (_0_) と正の値 (_1_) を切り替えることで、このプロセスをよりスムーズに実行できます。これによりモデルをより簡単にチューニングできます。

In [ ]:
df = df.replace({0:1, 1:0})
df['isPositive'].value_counts()

欠損値の数を確認します。

In [ ]:
df.isna().sum()

テキストフィールドには欠損値があります。通常、これらの欠損値をどう処理するかを決定します。データを削除するか、標準テキストを入力できます。

## 3.テキスト処理: ストップワードを削除し語幹解釈を実行する
([先頭に戻る](#Lab-2.1:-Applying-ML-to-an-NLP-Problem))

このタスクでは、ストップワードを削除し、テキストデータのステミングを実行します。データを正規化し、処理する必要のあるさまざまな情報の量を減らします。

[nltk](https://www.nltk.org/) は、人間の言語データの処理によく使われるプラットフォームです。分類、トークン化、語幹解釈、タグ付け、構文解析、セマンティック推論のためのテキストを処理するインターフェイスと関数が用意されています。

インポートしたら、必要な機能のみをダウンロードできます。この例では次を使います。

- **punkt** は文トークナイザです
- **stopwords** は使用できるストップワードのリストを提供します。

In [ ]:
# Install the library and functions
import nltk
nltk.download('punkt')
nltk.download('stopwords')

次のセクションでは、ストップワードを削除してテキストをクリーニングするプロセスを作成します。自然言語ツールキット (NLTK) ライブラリには、一般的なストップワードのリストが用意されています。このリストを使いますが、まずはリストから一部の単語を削除します。テキスト内に保持するストップワードは、感情を判別するのに役立ちます。

In [ ]:
import nltk, re
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize

# Get a list of stopwords from the NLTK library
stop = stopwords.words('english')

# These words are important for your problem. You don't want to remove them.
excluding = ['against', 'not', 'don', 'don\'t','ain', 'are', 'aren\'t', 'could', 'couldn\'t',
             'did', 'didn\'t', 'does', 'doesn\'t', 'had', 'hadn\'t', 'has', 'hasn\'t', 
             'have', 'haven\'t', 'is', 'isn\'t', 'might', 'mightn\'t', 'must', 'mustn\'t',
             'need', 'needn\'t','should', 'shouldn\'t', 'was', 'wasn\'t', 'were', 
             'weren\'t', 'won\'t', 'would', 'wouldn\'t']

# New stopword list
stopwords = [word for word in stop if word not in excluding]




snowball stemmer は単語を語幹にします。例えば、「walking」は「walk」という語幹になります。

In [ ]:
snow = SnowballStemmer('english')

データに対して他にもいくつかの正規化タスクを実行する必要があります。関数と働きは次のとおりです。

- 欠損値を空の文字列で置き換える
- テキストを小文字に変換する
- 先頭または末尾の空白を削除する
- 余分なスペースとタブを削除する
- HTML マークアップを削除する

 `for` ループでは、__NOT__ 数値の単語、2 文字より長い単語、ストップワードのリストに含まれない単語はすべて保持され、返されます。

In [ ]:
def process_text(texts): 
    final_text_list=[]
    for sent in texts:
        
        # Check if the sentence is a missing value
        if isinstance(sent, str) == False:
            sent = ''
            
        filtered_sentence=[]
        
        sent = sent.lower() # Lowercase 
        sent = sent.strip() # Remove leading/trailing whitespace
        sent = re.sub('\s+', ' ', sent) # Remove extra space and tabs
        sent = re.compile('<.*?>').sub('', sent) # Remove HTML tags/markups:
        
        for w in word_tokenize(sent):
            # Applying some custom filtering here, feel free to try different things
            # Check if it is not numeric and its length>2 and not in stopwords
            if(not w.isnumeric()) and (len(w)>2) and (w not in stopwords):  
                # Stem and add to filtered list
                filtered_sentence.append(snow.stem(w))
        final_string = " ".join(filtered_sentence) # Final string of cleaned words
 
        final_text_list.append(final_string)
        
    return final_text_list

## 4.トレーニング、検証、テストデータを分割する
([先頭に戻る](#Lab-2.1:-Applying-ML-to-an-NLP-Problem))

この手順では、sklearn [__train_test_split()__](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) 関数を使って、データセットをトレーニング (80%)、検証 (10%)、テスト (10%) に分割します。

トレーニングデータを使用してモデルをトレーニングし、テストデータでモデルをテストします。モデルのトレーニング完了後、検証セットを使って、モデルの実データでの動作に関するメトリクスを取得します。

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(df[['reviewText', 'summary', 'time', 'log_votes']],
                                                  df['isPositive'],
                                                  test_size=0.20,
                                                  shuffle=True,
                                                  random_state=324
                                                 )

X_val, X_test, y_val, y_test = train_test_split(X_val,
                                                y_val,
                                                test_size=0.5,
                                                shuffle=True,
                                                random_state=324)

データセットを分割することで、トレーニングセット、テストセット、検証セットの各テキストの特徴に対して上で定義した  `process_text` 関数を実行できるようになります。

In [ ]:
print('Processing the reviewText fields')
X_train['reviewText'] = process_text(X_train['reviewText'].tolist())
X_val['reviewText'] = process_text(X_val['reviewText'].tolist())
X_test['reviewText'] = process_text(X_test['reviewText'].tolist())

print('Processing the summary fields')
X_train['summary'] = process_text(X_train['summary'].tolist())
X_val['summary'] = process_text(X_val['summary'].tolist())
X_test['summary'] = process_text(X_test['summary'].tolist())

## 5.パイプラインと ColumnTransformer でデータを処理する
([先頭に戻る](#Lab-2.1:-Applying-ML-to-an-NLP-Problem))

多くの場合、データを使ってモデルをトレーニングする前に、データに対して多くのタスクを実行します。こうしたステップは、モデルのデプロイ後の推論に使われるすべてのデータに対しても実行する必要があります。これらのステップを整理するには、_pipeline_ を定義することをお勧めします。パイプラインはデータに対して実行される一連の処理タスクです。複数のパイプラインを作成して異なるフィールドを処理できます。テキストデータと数値データの両方を操作しているため、次のパイプラインを定義できます。

   * 数値特徴のパイプラインの場合、__numerical_processor__ は MinMaxScaler を使います。(決定木を使う場合は特徴をスケールする必要はありませんが、より多くのデータを変換する方法を確認することをお勧めします)。 複数の数値特徴に対して異なるタイプの処理を実行する場合は、2 つのテキスト特徴に表示されるパイプラインのように、複数のパイプラインを構築する必要があります。
   * テキスト特徴のパイプラインの場合、__text_processor__ はテキストフィールドに `CountVectorizer()` を使用します。
   
データセット特徴の選択的前処理は、一連の ColumnTransformer にまとめられ、パイプラインで推定器とともに使用されます。このプロセスにより、モデルを適合させる際や予測を行う際に、raw データに対して変換が自動で確実に実行されます。(例えば、交差検証によって検証データセットのモデルを評価する場合や、将来、テストデータセットで予測を行う場合)。

In [ ]:
# Grab model features/inputs and target/output
numerical_features = ['time',
                      'log_votes']

text_features = ['summary',
                 'reviewText']

model_features = numerical_features + text_features
model_target = 'isPositive'

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

### COLUMN_TRANSFORMER ###
##########################

# Preprocess the numerical features
numerical_processor = Pipeline([
    ('num_imputer', SimpleImputer(strategy='mean')),
    ('num_scaler', MinMaxScaler()) 
                                ])
# Preprocess 1st text feature
text_processor_0 = Pipeline([
    ('text_vect_0', CountVectorizer(binary=True, max_features=50))
                                ])

# Preprocess 2nd text feature (larger vocabulary)
text_precessor_1 = Pipeline([
    ('text_vect_1', CountVectorizer(binary=True, max_features=150))
                                ])

# Combine all data preprocessors from above (add more, if you choose to define more!)
# For each processor/step specify: a name, the actual process, and finally the features to be processed
data_preprocessor = ColumnTransformer([
    ('numerical_pre', numerical_processor, numerical_features),
    ('text_pre_0', text_processor_0, text_features[0]),
    ('text_pre_1', text_precessor_1, text_features[1])
                                    ]) 

### DATA PREPROCESSING ###
##########################

print('Datasets shapes before processing: ', X_train.shape, X_val.shape, X_test.shape)

X_train = data_preprocessor.fit_transform(X_train).toarray()
X_val = data_preprocessor.transform(X_val).toarray()
X_test = data_preprocessor.transform(X_test).toarray()

print('Datasets shapes after processing: ', X_train.shape, X_val.shape, X_test.shape)

データセット内の特徴の数が 4 から 202 にどのように増加したかに注意してください。

In [ ]:
print(X_train[0])

## 6.組み込み SageMaker アルゴリズムで分類器をトレーニングする
([先頭に戻る](#Lab-2.1:-Applying-ML-to-an-NLP-Problem))

このステップでは、次のオプションを指定して SageMaker `LinearLearner()` アルゴリズムを呼び出します。
* __Permissions-__ `role` は、現在の環境の AWS Identity and Access Management (IAM) ロールに設定します。
* __Compute power -__  `train_instance_count` パラメータと `train_instance_type` パラメータを使用します。この例では、 `ml.m4.xlarge` リソースをトレーニングに使用します。インスタンスタイプは必要に応じて変更できます。(例えば、GPU をニューラルネットワークに使用できます)。 
* __Model type -__ `predictor_type` は、今はバイナリ分類問題を処理するため、__`binary_classifier`__ に設定します。3 つ以上のクラスが関係している場合、__`multiclass_classifier`__ を使用できます。また、回帰問題には __`regressor`__ を使用できます。


In [ ]:
import sagemaker

# Call the LinearLearner estimator object
linear_classifier = sagemaker.LinearLearner(role=sagemaker.get_execution_role(),
                                           instance_count=1,
                                           instance_type='ml.m4.xlarge',
                                           predictor_type='binary_classifier')

推定器のトレーニング、検証、テストの各部分を設定するには、 `binary_estimator` の `record_set()` 関数を使用できます。

In [ ]:
train_records = linear_classifier.record_set(X_train.astype('float32'),
                                            y_train.values.astype('float32'),
                                            channel='train')
val_records = linear_classifier.record_set(X_val.astype('float32'),
                                          y_val.values.astype('float32'),
                                          channel='validation')
test_records = linear_classifier.record_set(X_test.astype('float32'),
                                           y_test.values.astype('float32'),
                                           channel='test')

 `fit()` 関数には、確率的勾配降下 (SGD) アルゴリズムの分散されたバージョンが適用されます。そこにデータを送信します。ログは `logs=False` で無効化されました。このパラメータを削除するとプロセスの詳細を表示できます。__このプロセスは ml.m4.xlarge インスタンスで約 3～4 分かかります。__

In [ ]:
linear_classifier.fit([train_records,
                       val_records,
                       test_records],
                       logs=False)

## 7.モデルを評価する
([先頭に戻る](#Lab-2.1:-Applying-ML-to-an-NLP-Problem))

SageMaker の分析を使用して、テストセットの (選択した) パフォーマンスメトリクスを取得できます。このプロセスではモデルをデプロイする必要はありません。

線形学習者によってトレーニング中に計算されるメトリクスが作成されます。このメトリクスはモデルをチューニングする際に使用できます。検証セットで使用できるメトリクスは次のとおりです。

- objective_loss - バイナリ分類問題の場合、これは各エポックのロジスティック損失の平均値になる
- binary_classification_accuracy - データセット上の最終モデルの精度。つまりモデルが適切に取得した予測の数
- precision - 真の陽性である陽性クラスの予測の数を定量化する
- recall - 陽性クラスの予測の数を定量化する
- binary_f_beta - 精度メトリクスと再現率メトリクスの調和平均

この例では、正しい予測の数に関心があります。**binary_classification_accuracy** メトリクスを使用するのが適切だと考えられます。

In [ ]:
sagemaker.analytics.TrainingJobAnalytics(linear_classifier._current_job_name, 
                                         metric_names = ['test:binary_classification_accuracy']
                                        ).dataframe()

0.85 前後の値が表示されます。値が異なる場合がありますが、この前後の値です。これは、モデルが 85% の時間で正解を正確に予測していることを意味します。ビジネスケースによっては、ハイパーパラメータのチューニングジョブを使用してモデルをさらにチューニングしたり、特徴量エンジニアリングをさらに実行したりする必要があります。

## 8.モデルをエンドポイントにデプロイする
([先頭に戻る](#Lab-2.1:-Applying-ML-to-an-NLP-Problem))

この演習の最後のセクションでは、選択した別のインスタンスにモデルをデプロイします。このモデルは本番環境で使用できます。デプロイされたエンドポイントは、AWS Lambda や Amazon API Gateway など、他の AWS のサービスで使用できます。詳細については、次のチュートリアルを参照してください。[Call an Amazon SageMaker model endpoint using Amazon API Gateway and AWS Lambda](https://aws.amazon.com/blogs/machine-learning/call-an-amazon-sagemaker-model-endpoint-using-amazon-api-gateway-and-aws-lambda/)

モデルをデプロイするには、次のセルを実行します。_ml.t2.medium_、_ml.c4.xlarge_) など、さまざまなインスタンスタイプを使用できます。__このプロセスの完了にはしばらく時間がかかります (約 7～8 分)。__

In [ ]:

linear_classifier_predictor = linear_classifier.deploy(initial_instance_count = 1,
                                                       instance_type = 'ml.c5.large'
                                                      )

## 9.エンドポイントをテストする
([先頭に戻る](#Lab-2.1:-Applying-ML-to-an-NLP-Problem))

エンドポイントがデプロイされたので、テストデータをそのエンドポイントに送信し、データから予測を取得します。

In [ ]:
import numpy as np

# Get test data in batch size of 25 and make predictions.
prediction_batches = [linear_classifier_predictor.predict(batch)
                      for batch in np.array_split(X_test.astype('float32'), 25)
                     ]

# Get a list of predictions
print([pred.label['score'].float32_tensor.values[0] for pred in prediction_batches[0]])

## 10.モデルアーティファクトをクリーンアップする
([先頭に戻る](#Lab-2.1:-Applying-ML-to-an-NLP-Problem))

エンドポイントを使用後、次を実行して削除できます。

**ヒント:** - ご自身のアカウントを使用する場合、エンドポイントやその他のリソースを削除しないと料金が発生することに注意してください。

In [ ]:
linear_classifier_predictor.delete_endpoint()

# お疲れ様でした。

このラボでは非常にシンプルな NLP の問題について確認しました。ラベル付きデータセットを使用し、シンプルなトークナイザとエンコーダで線形学習者モデルのトレーニングに必要なデータを生成しました。次に、モデルをデプロイし、いくつかの予測を実行しました。この作業を実際に行っている場合、トレーニング用にデータを取得し、ラベル付けをする必要があります。または、事前トレーニング済みアルゴリズムまたはマネージドサービスを使用することもできます。ハイパーパラメータのチューニングジョブを使用して、モデルをさらにチューニングすることもできます。

このラボを完了しました。ラボガイドの手順に従ってラボを終了してください。

©2023 Amazon Web Services, Inc. or its affiliates.All rights reserved.このトレーニング内容の全体または一部を複製または再配布することは、Amazon Web Services, Inc. の書面による事前の許可がある場合を除き、禁じられています。商業目的のコピー、貸与、販売を禁止します。すべての商標は各所有者に帰属します。